# Evaluation of RAG Using Ragas

In the following notebook we'll explore how to evaluate RAG pipelines using a powerful open-source tool called "Ragas". This will give us tools to evaluate component-wise metrics, as well as end-to-end metrics about the performance of our RAG pipelines.

In the following notebook we'll complete the following tasks:

The only way to get started is to get started - so let's grab our dependencies for the day!

> NOTE: Using this notebook as presented will incur a charge of ~$3USD from OpenAI usage.

- 🤝 Breakout Room #1
  1. Task 1: Installing Required Libraries
  2. Task 2: Set Environment Variables
  3. Task 3: Creating a Simple RAG Pipeline with LangChain v.0.2.0
  4. Task 4: Synthetic Dataset Generation for Evaluation using Ragas (Optional)

- 🤝 Breakout Room #2
  1. Task 1: Evaluating our Pipeline with Ragas
  2. Task 2: Testing OpenAI's Claim
  3. Task 3: Selecting an Advanced Retriever and Evaluating

> NOTE: This Notebook *does* contain a bonus challenge, outlined at the bottom of the notebook, which you can complete instead of the notebook for full marks on the assignment.

## Motivation

A claim, made by OpenAI, is that their `text-embedding-3-small` is better (generally) than their `text-embedding-ada-002` model.

Here's some passages from their [blog](https://openai.com/blog/new-embedding-models-and-api-updates) about the `text-embedding-3` release:

> `text-embedding-3-small` is our new highly efficient embedding model and provides a significant upgrade over its predecessor, the `text-embedding-ada-002` model...

> **Stronger performance.** Comparing `text-embedding-ada-002` to `text-embedding-3-small`, the average score on a commonly used benchmark for multi-language retrieval ([MIRACL](https://github.com/project-miracl/miracl)) has increased from 31.4% to 44.0%, while the average score on a commonly used benchmark for English tasks ([MTEB](https://github.com/embeddings-benchmark/mteb)) has increased from 61.0% to 62.3%.

Well, with a library like Ragas - we can put that claim to the test!

If what they claim is true - we should see an increase on related metrics by using the new embedding model!

# 🤝 Breakout Room Part #1

## Task 1: Installing Required Libraries

A reminder that one of the [key features](https://python.langchain.com/v0.2/docs/versions/v0_2/) of LangChain v0.2.0 is the compartmentalization of the various LangChain ecosystem packages!

So let's begin grabbing all of our LangChain related packages!

In [1]:
!pip install -U -q langchain langchain-openai langchain_core langchain-community langchainhub openai langchain-qdrant

We'll also get the "star of the show" today, which is Ragas!

In [3]:
!pip install -qU ragas

We'll be leveraging [QDrant](https://qdrant.tech/) again as our LangChain `VectorStore`.

We'll also install `pymupdf` and its dependencies which will allow us to load PDFs using the `PyMuPDFLoader` in the `langchain-community` package!

In [2]:
!pip install -qU qdrant-client pymupdf pandas

## Task 2: Set Environment Variables

Let's set up our OpenAI API key so we can leverage their API later on.

In [4]:
import os
import openai
from getpass import getpass

# openai.api_key = getpass("Please provide your OpenAI Key: ")
# os.environ["OPENAI_API_KEY"] = openai.api_key

## Task 3: Creating a Simple RAG Pipeline with LangChain v0.2.0

Building on what we've been learning, we'll be leveraging LangChain v0.2.0 and LCEL to build a simple RAG pipeline that we can baseline with Ragas.

## Building our RAG pipeline

Let's review the basic steps of RAG again:

- Create an Index
- Use retrieval to obtain pieces of context from our Index that are similar to our query
- Use a LLM to generate responses based on the retrieved context

Let's get started by creating our index.

> NOTE: We're going to start leaning on the term "index" to refer to our `VectorStore`, `VectorDatabase`, etc. We can think of "index" as the catch-all term, whereas `VectorStore` and the like relate to the specific technologies used to create, store, and interact with the index.

### Creating an Index

You'll notice that the largest changes (outside of some import changes) are that our old favourite chains are back to being bundled in an easily usable abstraction.

We can still create custom chains using LCEL - but we can also be more confident that our pre-packaged chains are creating using LCEL under the hood.

#### Loading Data

Let's start by loading some data!

- [`PyMuPDFLoader`](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.pdf.PyMuPDFLoader.html)

> NOTE: You'll notice that we're using a document loader from the community package of LangChain. This is part of the v0.2.0 changes that make the base (`langchain-core`) package remain lightweight while still providing access to some of the more powerful community integrations.

In [5]:
from langchain_community.document_loaders import PyMuPDFLoader

PDF_LINK = "https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf"

loader = PyMuPDFLoader(
    PDF_LINK  ### INSERT CODE
)

documents = loader.load()### YOUR CODE HERE

In [6]:
documents[0].metadata

{'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf',
 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf',
 'page': 0,
 'total_pages': 195,
 'format': 'PDF 1.3',
 'title': 'The Pmarca Blog Archives',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': '',
 'producer': 'Mac OS X 10.10 Quartz PDFContext',
 'creationDate': "D:20150110020418Z00'00'",
 'modDate': "D:20150110020418Z00'00'",
 'trapped': ''}

#### Transforming Data

Now that we've got our single document - let's split it into smaller pieces so we can more effectively leverage it with our retrieval chain!

We'll start with the classic: `RecursiveCharacterTextSplitter`.

- [`RecursiveCharacterTextSplitter`](https://api.python.langchain.com/en/latest/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html#langchain-text-splitters-character-recursivecharactertextsplitter)

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

CHUNK_SIZE = 200
CHUNK_OVERLAP = 50

text_splitter = RecursiveCharacterTextSplitter(
    ### YOUR CODE HERE
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP
)

documents = text_splitter.split_documents(documents)  ### YOUR CODE HERE

Let's confirm we've split our document.

In [9]:
len(documents)

1864

#### Loading OpenAI Embeddings Model

We'll need a process by which we can convert our text into vectors that allow us to compare to our query vector.

Let's use OpenAI's `text-embedding-ada-002` for this task!

- [`OpenAIEmbeddings`](https://api.python.langchain.com/en/latest/embeddings/langchain_openai.embeddings.base.OpenAIEmbeddings.html#langchain-openai-embeddings-base-openaiembeddings)

> NOTE: We are purposefully using an older embedding model to try and answer the guiding question: Is TE3 better than Ada-002?

In [10]:
from langchain_openai import OpenAIEmbeddings

EMBEDDING_MODEL = "text-embedding-ada-002"

embeddings = OpenAIEmbeddings(
    model = EMBEDDING_MODEL  ### YOUR CODE HERE
)

#### Creating a QDrant VectorStore

Now that we have documents - we'll need a place to store them alongside their embeddings.

- [`Qdrant`](https://api.python.langchain.com/en/latest/qdrant/langchain_qdrant.qdrant.QdrantVectorStore.html#langchain_qdrant.qdrant.QdrantVectorStore)

> NOTE: You'll need to provide the embedding dimension for Ada-002!

In [18]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

LOCATION = ":memory:"
COLLECTION_NAME = "PMarca Blogs"
VECTOR_SIZE = 1536

In [21]:
qdrant_client = QdrantClient(LOCATION)  ### YOUR CODE HERE

qdrant_client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=VectorParams(size=VECTOR_SIZE,distance=Distance.COSINE)
    
    ### YOUR CODE HERE
)

True

In [23]:


qdrant_vector_store = QdrantVectorStore(
    client=qdrant_client,
    collection_name=COLLECTION_NAME,
    embedding=embeddings



    ### YOUR CODE HERE
)

qdrant_vector_store.add_documents(documents)

['83b274c26f674ff79b8bcb4238538a13',
 '43101eaa2e1e4f2c9ae60048ad0d1f36',
 'd3ea5d87d4cd41f3b4f8a841dccd1ea4',
 '5799febe09204bd4b73761668e320683',
 'f237df6db3074466ad9cce8c4eea5b69',
 '171f6ef0b08341fcaa5a133951688f64',
 '9c13bd19c7064d59a72a5dd68524841b',
 '1dd5aff9201546d2b440de4bab429b85',
 '9255224729e8472eba4f4757fe602e8c',
 '0038f804e72e452cbf926ffccd6cba7b',
 '2ecb239c3c37438da9981082f6663bf4',
 '1b5f8fbe2e694e76845f476e913ebfe1',
 'ac3b3a5f7fae4c60b436038f0fc8bdeb',
 '41f9c991577b4ebc93861b35c4aed1b4',
 'ea573a358e604319ba6cefe7283f225b',
 '74e38e567c184abe9d1e8e67ef2e9a68',
 'ca5c31cf679e408b8325a09c6aa5ff21',
 'f8e00960afa14a93b0cfa757ddf42fb3',
 '513766e839664d608e9d0a1c41b4fba0',
 '7d5ac7449e154e16acac8a63548567b0',
 'd6bcbe409c984ea395c27d1fb59ef1f1',
 'caa1d3c46a66498f8869e0ed35e6d422',
 'f501844af71b4b42b215f7a7bd6dfe3b',
 '10bdc0831e3a4ae3af386af0fc28d4ed',
 'f2551f315da342d2bac90986d44cd6d9',
 '5f72338219d74cb394f86552065ebdd9',
 '2ae1902e72fd4a05a5963e17de943d2c',
 

####❓ Question #1:

List out a few of the techniques that Qdrant uses that make it performant.

> NOTE: Check the [documentation](https://qdrant.tech/documentation/overview/) for more information about QDrant!

#### Creating a Retriever

To complete our index, all that's left to do is expose our vectorstore as a retriever - which we can do the same way we would in previous version of LangChain!

In [24]:
retriever = qdrant_vector_store.as_retriever()  ### YOUR CODE HERE

#### Testing our Retriever

Now that we've gone through the trouble of creating our retriever - let's see it in action!

In [25]:
retrieved_documents = retriever.invoke("What is a rule of thumb for selecting an industry to invest in?")

Failed to batch ingest runs: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/batch. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/batch', '{"detail":"Monthly unique traces usage limit exceeded"}')
post: trace=80b5b2fd-d116-4b28-bac3-bb23a47f458d,id=80b5b2fd-d116-4b28-bac3-bb23a47f458d; trace=42e8fa81-3df6-47ca-98b1-051a4399b232,id=42e8fa81-3df6-47ca-98b1-051a4399b232; trace=f3db7ccd-79f5-47e4-b691-190e8c91e43b,id=f3db7ccd-79f5-47e4-b691-190e8c91e43b; trace=93e82ded-6108-4e0b-b46c-14ccb9edb50f,id=93e82ded-6108-4e0b-b46c-14ccb9edb50f; trace=219dd5a9-326d-46bb-8b1e-c8565a33478e,id=219dd5a9-326d-46bb-8b1e-c8565a33478e; trace=0cf0e368-c39b-4391-b4ac-823763b3f064,id=0cf0e368-c39b-4391-b4ac-823763b3f064; trace=053575b0-7e3c-4dfd-8574-78a92cd5c1f3,id=053575b0-7e3c-4dfd-8574-78a92cd5c1f3; trace=a67d3f33-11c2-4381-9608-0f3b4ef9d217,id=a67d3f33-11c2-4381-9608-0f3b4ef9d217; trace=115efc3a-9348-427a

In [26]:
for doc in retrieved_documents:
  print(doc)

page_content='the existing order — and make sure that those forces of change
have a reasonable chance at succeeding.
Second rule of thumb:
Once you have picked an industry, get right to the center of it' metadata={'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'page': 125, 'total_pages': 195, 'format': 'PDF 1.3', 'title': 'The Pmarca Blog Archives', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Mac OS X 10.10 Quartz PDFContext', 'creationDate': "D:20150110020418Z00'00'", 'modDate': "D:20150110020418Z00'00'", 'trapped': '', '_id': 'e20ac675f61249eeb2f5a17939072f9b', '_collection_name': 'PMarca Blogs'}
page_content='Third rule:
In a rapidly changing Held like technology, the best place to
get experience when you’re starting out is in younger, high-
growth companies.' metadata={'source': 'https://d1lamhf6l6yk6d.cloud

### Creating a RAG Chain

Now that we have the "R" in RAG taken care of - let's look at creating the "AG"!

#### Creating a Prompt Template

There are a few different ways we could create our prompt template - we could create a custom template, as seen in the code below, or we could simply pull a prompt from the prompt hub! Let's look at an example of that!

In [27]:
from langchain import hub

retrieval_qa_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

/Users/jmichaeldean/anaconda3/envs/AIE4a/lib/python3.11/site-packages/langsmith/client.py:5489: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))


In [28]:
print(retrieval_qa_prompt.messages[0].prompt.template)

Answer any use questions based solely on the context below:

<context>
{context}
</context>


As you can see - the prompt template is simple (and has a small error) - so we'll create our own to be a bit more specific!

In [29]:
from langchain.prompts import ChatPromptTemplate

template = """
You are a helpful assistant.  Answer the question based on information in the context.  If you cannot answer the
question then say you do not know.

Question:
{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_template(template)   ### YOUR CODE HERE

#### Setting Up our Basic QA Chain

Now we can instantiate our basic RAG chain!

We'll use LCEL directly just to see an example of it - but you could just as easily use an abstraction here to achieve the same goal!

We'll also ensure to pass-through our context - which is critical for RAGAS.

In [30]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

primary_qa_llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

Let's test it out!

In [31]:
question = "What is a rule of thumb for selecting an industry to invest in?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)

A rule of thumb for selecting an industry to invest in is to ensure that the forces of change within that industry have a reasonable chance of succeeding. Additionally, once you have picked an industry, it is advised to get right to the center of it, where the great opportunities can be found.


In [32]:
question = "What did Pink Floyd have to say about how to proceed when investing in a new industry?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)
print(result["context"])

I do not know.
[Document(metadata={'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'page': 15, 'total_pages': 195, 'format': 'PDF 1.3', 'title': 'The Pmarca Blog Archives', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Mac OS X 10.10 Quartz PDFContext', 'creationDate': "D:20150110020418Z00'00'", 'modDate': "D:20150110020418Z00'00'", 'trapped': '', '_id': '6e43ee8782ff4e54adab84a4da670930', '_collection_name': 'PMarca Blogs'}, page_content='ask if you can call them again if things change.\nTrust me — they’d much rather be saying “yes” than “no” —\nthey need all the good investments they can get.\nSecond, consider the environment.'), Document(metadata={'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The

We can already see that there are some improvements we could make here.

For now, let's switch gears to RAGAS to see how we can leverage that tool to provide us insight into how our pipeline is performing!

## Task 4: Synthetic Dataset Generation for Evaluation using Ragas

Ragas is a powerful library that lets us evaluate our RAG pipeline by collecting input/output/context triplets and obtaining metrics relating to a number of different aspects of our RAG pipeline.

We'll be evaluating on every core metric today, but in order to do that - we'll need to create a test set. Luckily for us, Ragas can do that directly!

### Synthetic Test Set Generation

We can leverage Ragas' [`Synthetic Test Data generation`](https://docs.ragas.io/en/stable/concepts/testset_generation.html) functionality to generate our own synthetic QC pairs - as well as a synthetic ground truth - quite easily!

In [33]:
loader = PyMuPDFLoader(
    "https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf",
)

eval_documents = loader.load()

text_splitter_eval = RecursiveCharacterTextSplitter(
    chunk_size = 600,
    chunk_overlap = 50
)

eval_documents = text_splitter_eval.split_documents(eval_documents)

####❓ Question #2:

Why is it important to split our documents using different parameters when creating our synthetic data?

In [34]:
len(eval_documents)

624

> NOTE: 🛑 Running this cell as presented will incur a charge of ~$3USD from OpenAI usage. Most of this cost is produced by the Synthetic Data Generation step. **YOU CAN SKIP THIS STEP BY LOADING THE `.csv` DIRECTLY FROM OUR REPOSITORY.** 🛑

#### Optional: SDG for Evaluation

In [35]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

generator_llm = ChatOpenAI(model="gpt-4o-mini")
critic_llm = ChatOpenAI(model="gpt-4o")
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

distributions = {
    simple: 0.5,
    multi_context: 0.4,
    reasoning: 0.1
}

num_qa_pairs = 20 # You can reduce the number of QA pairs to 5 if you're experiencing rate-limiting issues

testset = generator.generate_with_langchain_docs(eval_documents, num_qa_pairs, distributions)
testset.to_pandas()

embedding nodes:   0%|          | 0/1248 [00:00<?, ?it/s]

Filename and doc_id are the same for all nodes.


Generating:   0%|          | 0/20 [00:00<?, ?it/s]

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What are the four major forms of chance and th...,[that chance is immune from human intervention...,The answer to given question is not present in...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
1,What is the purpose of temporary subfolders in...,[you can reply to a lot of messages with “I’m ...,Temporary subfolders in email management are u...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
2,What impact does a severe personality disorder...,[added humor value.]\nWhile I enjoyed Marc’s p...,The answer to given question is not present in...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
3,What is the relationship between precocious in...,[These three components are conspicuously link...,The context suggests that precocious individua...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
4,What are some strategies for raising angel mon...,[This obviously raises the issue of how you’re...,The context suggests several strategies for ra...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
5,What is the Influence-from-Mere-Association Te...,[One very practical consequence of Liking/Lovi...,The Influence-from-Mere-Association Tendency r...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
6,What should individuals focus on to succeed in...,[becomes irrelevant to determining the success...,"According to Dr. Simonton, individuals should ...",simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
7,What is the significance of being an individua...,[from scratch. This is a sharp diWerence from ...,Being an individual contributor is significant...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
8,What makes engineering degrees considered usef...,[workforce in a high-impact way when you gradu...,Engineering degrees are considered useful in t...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
9,What services are people abandoning in favor o...,[billion people online now. That is a very lar...,People are abandoning older services like news...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True


Let's look at the output and see what we can learn about it!

In [36]:
testset.test_data[0]

DataRow(question='What are the four major forms of chance and their distinct roles in human interactions?', contexts=['that chance is immune from human interventions. However, one\nmust be careful not to read any unconsciously purposeful intent\ninto “interventions”… [which] are to be viewed as accidental,\nunwilled, inadvertent, and unforseeable.\nIndeed, chance plays several distinct roles when humans react cre-\natively with one another and with their environment…\nWe can observe chance arriving in four major forms and for four\ndiWerent reasons. The principles involved aWect everyone.\nHere’s where it helps to be a neurologist writing on this topic:\nThe four kinds of chance each have a diWerent kind of motor'], ground_truth='The answer to given question is not present in context', evolution_type='simple', metadata=[{'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The

In [37]:
testset_df = testset.to_pandas()
testset_df.to_csv("testset.csv")

#### PREFERRED: Download `.csv` from DataRepository

I have to consume $85 before the end of September because credits I bought a year ago are going to expire!

In [ ]:
# !git clone https://github.com/AI-Maker-Space/DataRepository.git

Cloning into 'DataRepository'...
remote: Enumerating objects: 87, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 87 (delta 24), reused 28 (delta 8), pack-reused 8 (from 1)
Receiving objects: 100% (87/87), 70.09 MiB | 33.73 MiB/s, done.
Resolving deltas: 100% (24/24), done.


In [ ]:
# !mv DataRepository/testset.csv .

### Generating Responses with RAG Pipeline

Now that we have some QC pairs, and some ground truths, let's evaluate our RAG pipeline using Ragas.

The process is, again, quite straightforward - thanks to Ragas and LangChain!

Let's start by extracting our questions and ground truths from our create testset.

We can start by converting our test dataset into a Pandas DataFrame.

In [38]:
import pandas as pd

test_df = pd.read_csv("testset.csv")

In [39]:
test_df

,Unnamed: 0,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,0,What are the four major forms of chance and th...,['that chance is immune from human interventio...,The answer to given question is not present in...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
1,1,What is the purpose of temporary subfolders in...,['you can reply to a lot of messages with “I’m...,Temporary subfolders in email management are u...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
2,2,What impact does a severe personality disorder...,['added humor value.]\nWhile I enjoyed Marc’s ...,The answer to given question is not present in...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
3,3,What is the relationship between precocious in...,['These three components are conspicuously lin...,The context suggests that precocious individua...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
4,4,What are some strategies for raising angel mon...,['This obviously raises the issue of how you’r...,The context suggests several strategies for ra...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
5,5,What is the Influence-from-Mere-Association Te...,['One very practical consequence of Liking/Lov...,The Influence-from-Mere-Association Tendency r...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
6,6,What should individuals focus on to succeed in...,['becomes irrelevant to determining the succes...,"According to Dr. Simonton, individuals should ...",simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
7,7,What is the significance of being an individua...,['from scratch. This is a sharp diWerence from...,Being an individual contributor is significant...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
8,8,What makes engineering degrees considered usef...,['workforce in a high-impact way when you grad...,Engineering degrees are considered useful in t...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
9,9,What services are people abandoning in favor o...,['billion people online now. That is a very la...,People are abandoning older services like news...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True


In [40]:
test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()

Now we'll generate responses using our RAG pipeline using the questions we've generated - we'll also need to collect our retrieved contexts for each question.

We'll do this in a simple loop to see exactly what's happening!

In [41]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_augmented_qa_chain.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])

Now we can wrap our information in a Hugging Face dataset for use in the Ragas library.

In [42]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's take a peek and see what that looks like!

In [43]:
response_dataset[0]

{'question': 'What are the four major forms of chance and their distinct roles in human interactions?',
 'answer': 'The four major forms of chance mentioned in the context are unwilled, inadvertent, and unforeseeable. Each of these forms plays distinct roles in human interactions, involving different kinds of exploratory activity and sensory receptivity. However, the specific details about the four forms of chance and their distinct roles are not provided in the context. Therefore, I cannot provide a complete answer regarding the four major forms of chance and their distinct roles.',
 'contexts': ['We can observe chance arriving in four major forms and for four\ndiWerent reasons. The principles involved aWect everyone.\nHere’s where it helps to be a neurologist writing on this topic:',
  'unwilled, inadvertent, and unforseeable.\nIndeed, chance plays several distinct roles when humans react cre-\natively with one another and with their environment…',
  'The four kinds of chance each ha

# 🤝 Breakout Room Part #2

## Task 1: Evaluating our Pipeline with Ragas

Now that we have our response dataset - we can finally get into the "meat" of Ragas - evaluation!

First, we'll import the desired metrics, then we can use them to evaluate our created dataset!

Check out the specific metrics we'll be using in the Ragas documentation:

- [Faithfulness](https://docs.ragas.io/en/stable/concepts/metrics/faithfulness.html)
- [Answer Relevancy](https://docs.ragas.io/en/stable/concepts/metrics/answer_relevance.html)
- [Context Precision](https://docs.ragas.io/en/stable/concepts/metrics/context_precision.html)
- [Context Recall](https://docs.ragas.io/en/stable/concepts/metrics/context_recall.html)
- [Answer Correctness](https://docs.ragas.io/en/stable/concepts/metrics/answer_correctness.html)

See the accompanied presentation for more in-depth explanations about each of the metrics!

In [44]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

All that's left to do is call "evaluate" and away we go!

In [45]:
results = evaluate(response_dataset, metrics)

Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

In [46]:
results

{'faithfulness': 0.6709, 'answer_relevancy': 0.7698, 'context_recall': 0.7500, 'context_precision': 0.6972, 'answer_correctness': 0.6469}

In [47]:
results_df = results.to_pandas()
results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,What are the four major forms of chance and th...,The four major forms of chance mentioned in th...,[We can observe chance arriving in four major ...,The answer to given question is not present in...,0.600000,0.000000,1.000000,0.000000,0.188577
1,What is the purpose of temporary subfolders in...,The purpose of temporary subfolders in email m...,"[the normal course of your day.\nFourth, aside...",Temporary subfolders in email management are u...,0.500000,1.000000,1.000000,1.000000,0.994009
2,What impact does a severe personality disorder...,A severe personality disorder can lead a manag...,[severe personality disorder who micromanages ...,The answer to given question is not present in...,0.600000,1.000000,1.000000,0.000000,0.179351
3,What is the relationship between precocious in...,The relationship between precocious individual...,"[early, end late, and produce at above-average...",The context suggests that precocious individua...,0.857143,0.983589,1.000000,1.000000,0.765158
4,What are some strategies for raising angel mon...,Some strategies for raising angel money before...,[This obviously raises the issue of how you’re...,The context suggests several strategies for ra...,0.700000,1.000000,0.500000,0.805556,0.862922
5,What is the Influence-from-Mere-Association Te...,I do not know.,[The Psychology of Entrepreneurial Misjudgment...,The Influence-from-Mere-Association Tendency r...,0.000000,0.000000,0.000000,0.000000,0.182111
6,What should individuals focus on to succeed in...,"According to Dr. Simonton, individuals should ...",[progress through a creative career. Instead y...,"According to Dr. Simonton, individuals should ...",1.000000,0.981602,1.000000,1.000000,0.804426
7,What is the significance of being an individua...,The significance of being an individual contri...,[destiny — you get to succeed or fail on your ...,Being an individual contributor is significant...,0.444444,1.000000,0.666667,1.000000,0.843277
8,What makes engineering degrees considered usef...,Engineering degrees are considered useful in t...,[Which undergraduate degrees are useful in\nth...,Engineering degrees are considered useful in t...,0.857143,0.987121,1.000000,1.000000,0.539575
9,What services are people abandoning in favor o...,The context does not specify the exact service...,[billion people online now. That is a very lar...,People are abandoning older services like news...,0.750000,0.000000,1.000000,1.000000,0.407606


## Task : Testing OpenAI's Claim

Now that we've seen how our retriever can impact the performance of our RAG pipeline - let's see how changing our embedding model impacts performance.

####🏗️ Activity #1:

Please provide markdown, or code comments, to explain which each of the following steps are doing!

In [48]:
te3_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [49]:
qdrant_client.create_collection(
    collection_name=COLLECTION_NAME+"TE3",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

qdrant_vector_store = QdrantVectorStore(
    client=qdrant_client,
    collection_name=COLLECTION_NAME+"TE3",
    embedding=te3_embeddings,
)

qdrant_vector_store.add_documents(documents)

['b47e0fc002b849518c4ccc41626165a9',
 'b40a34304ab54f279f4b513793cbfdee',
 'f72c0d8a660e4b7ba394aa495a161b06',
 'cbe4570cc6bb479597addfadb552c244',
 '0d2f7d76517d47449b7e366a8d8ff82c',
 '95cb90ea5486428e85461c0fc5e420c7',
 'eba9bffe8765446cad2edf9a557fef84',
 'e5921ac63e474bc7b4675789d89862a9',
 'ed4fa26402f84c4587b569a48a7b8438',
 '9e4d2c09965747e0bf3b7c9d310171b3',
 '593ac9ee1c3a4bdebbe552797b858efa',
 'fa13350c34b640119d3c1393de20ff3f',
 'dc4b549d0d164929a3da5066434178c0',
 'c7615c4dabc04306a6ac2440d0126ef3',
 '35d02a055b2d4780b77fbf58adc9758e',
 '970b7e63bcf44d75bb1c8e47dd51ec20',
 '3d5ee7c86c0542ea8ff03d96a57b827b',
 '3138d56c78564a2194145e9a513832f0',
 'dfa89adfc0cb400195b5f98d6cb9b05c',
 'f12a8834338f45719c8ddce5ab9c48bf',
 '967e846771544605800719d2c74026a1',
 'a04681cbf6ed4f0d964146b20d8c79ad',
 '591d2bda03d5411a8bb388e55487fcfc',
 '837e4ba2e2324cdb9c12eb96f1bc7613',
 '2d614208225e4e5dae8d4f98e4361288',
 '06ccf0d3127a40d5928a1451444d61e4',
 'af48a9d50ab84406a087906a3d672cc3',
 

In [50]:
te3_retriever = qdrant_vector_store.as_retriever()

In [51]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(primary_qa_llm, retrieval_qa_prompt)

In [52]:
from langchain.chains import create_retrieval_chain

te3_retrieval_chain = create_retrieval_chain(te3_retriever, document_chain)

In [53]:
answers = []
contexts = []

for question in test_questions:
  response = te3_retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

In [54]:
te3_response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

In [55]:
te3_advanced_retrieval_results = evaluate(te3_response_dataset_advanced_retrieval, metrics)

Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

In [57]:
te3_advanced_retrieval_results

{'faithfulness': 0.8338, 'answer_relevancy': 0.7684, 'context_recall': 0.7000, 'context_precision': 0.7458, 'answer_correctness': 0.6128}

In [58]:
df_baseline = pd.DataFrame(list(results.items()), columns=['Metric', 'ADA'])
df_comparison = pd.DataFrame(list(te3_advanced_retrieval_results.items()), columns=['Metric', 'TE3'])

df_merged = pd.merge(df_baseline, df_comparison, on='Metric')

df_merged['Baseline -> TE3'] = df_merged['TE3'] - df_merged['ADA']

df_merged

,Metric,ADA,TE3,Baseline -> TE3
0,faithfulness,0.670853,0.833770,0.162917
1,answer_relevancy,0.769766,0.768356,-0.001410
2,context_recall,0.750000,0.700000,-0.050000
3,context_precision,0.697222,0.745833,0.048611
4,answer_correctness,0.646875,0.612831,-0.034044


####❓ Question #3:

Do you think, in your opinion, `text-embedding-3-small` is significantly better than `ada`?

## Task 5: Selecting an Advanced Retriever and Evaluating

#### 🏗️ Activity #2

While the changes that occured due to modifying the embedding model were desirable - you're now tasked with improving `context_recall`, or `context_precision` (or both!).

You'll follow these steps:

1. Reason about this list of Advanced Retrieval methods:
  - [Contextual Compression (Reranker)](https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/contextual_compression/)
  - [MultiQueryRetriever](https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/MultiQueryRetriever/)
  - [Parent Document Retriever](https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/parent_document_retriever/)
2. Select the method you think will be the most performant.
3. Implement that method.
4. Create a LCEL chain that utlizes the new Retriever method.
5. Evaluate this LCEL and compare to the TE3 results.

> NOTE: We will spend more time in Session 14 diving into advanced retrieval methods, this activity is only to serve as a basic introduction to the idea of component-wise improvements and how they might impact metrics.

In [ ]:
### YOUR CODE HERE

#### 🚧 BONUS CHALLENGE 🚧

> NOTE: Completing this challenge will provide full marks on the assignment, regardless of the complete of the notebook. You do not need to complete this in the notebook for full marks.

##### **MINIMUM REQUIREMENTS**:

1. Baseline `LCEL RAG` Application using `NAIVE RETRIEVAL`
2. Baseline Evaluation using `RAGAS METRICS`
  - [Faithfulness](https://docs.ragas.io/en/stable/concepts/metrics/faithfulness.html)
  - [Answer Relevancy](https://docs.ragas.io/en/stable/concepts/metrics/answer_relevance.html)
  - [Context Precision](https://docs.ragas.io/en/stable/concepts/metrics/context_precision.html)
  - [Context Recall](https://docs.ragas.io/en/stable/concepts/metrics/context_recall.html)
  - [Answer Correctness](https://docs.ragas.io/en/stable/concepts/metrics/answer_correctness.html)
3. Implement a `SEMANTIC CHUNKING STRATEGY`.
4. Create an `LCEL RAG` Application using `SEMANTIC CHUNKING` with `NAIVE RETRIEVAL`.
5. Compare and contrast results.

##### **SEMANTIC CHUNKING REQUIREMENTS**:

Chunk semantically similar (based on designed threshold) sentences, and then paragraphs, greedily, up to a maximum chunk size. Minimum chunk size is a single sentence.

Have fun!